# Задание

Реализовать классический метод ветвей и границ, в котором нужно ветвиться по дробным переменным.

# Настройки/Гиперпараметры/Импорты

In [1]:
import numpy as np # для быстрой работы с массивами
import random # для рандомизированного алгоритма
import time # для подсчёта времени работы
from tqdm import tqdm # для обёртки итераций между датасетами
import csv # для сохранения ответов
import pandas as pd # для вывода таблицы (v 2.0.0)

from docplex.mp.model import Model # импортирование cplex модели-функции

In [2]:
runs = 5 # число запусков для усреднения времени
iterations = 1000 # сколько попыток делать для поиска хорошего начального решения

## Вспомогательные функции

In [3]:
def check_solution(edges: dict, solution) -> list:
    """
    Функция для проверка полученного лучшего решения.\n
    Parameters:
        * edges: словарь смежных вершинам вершин, описывающий граф
        * solution: решение в формате (размер клики, [вершины в клике])\n
    Returns:
        * list — массив с ошибками (пуси, если ошибок нет)
    """

    errors = [] # массив для ошибок решения

    for i in range(int(solution[0])-1): # идём по вершинам и проверяем, смежны ли она со всеми остальными вершинами в найденной клике (solution[0] - число вершин в найденной клике в формате float)
        for j in range(i+1, int(solution[0])): # идём по последующим вершинам в клике
            if solution[1][j] not in edges[solution[1][i]]: # проверяем наличие ребра между вершинами
                errors.append([i, j]) # добавляем ошибку в список ошибок
                # raise RuntimeError("Clique contains unconnected vertices!") # если ребра нет — выкидываем ошибку
    return errors # возвращаем ошибки


# def check_solution_v2(edges: dict, solution, eps: float=0.000001) -> list:
#     """
#     Функция для проверка полученного лучшего решения.\n
#     Parameters:
#         * edges: словарь смежных вершинам вершин, описывающий граф
#         * solution: решение в формате (значение целевой функции, [значения всех переменных])
#         * eps: с какой погрешностью считать, что переменная целая\n
#     Returns:
#         * list — массив с ошибками (пуси, если ошибок нет)
#     """

#     errors = [] # массив для ошибок решения

#     for i in range(len(solution[1])-1): # идём по вершинам и проверяем, находятся ли они в клике и смежны ли она со всеми остальными вершинами в найденной клике (solution[1] - массив значений всех переменных, -1 чтобы не сравнивать с самим собой и не выйти за границы (у последнего элемента все комбинации уже будут рассмотрены))
#         if -eps <= solution[1][i] <= eps: # если вершина i не в клике (её значение очень близко к нулю) — переходим к рассмотрению следующей i
#             continue
#         for j in range(i+1, len(solution[1])): # идём по последующим переменным в решении
#             if (-eps <= solution[1][j] <= eps): # если вершина j не в клике (её значение очень близко к нулю) — переходим к рассмотрению следующей j
#                 continue # переходим к рассмотрению следующей пары
#             elif solution[1][j] not in edges[solution[1][i]]: # проверяем наличие ребра между вершинами (если в решении они обе в клике)
#                 errors.append([i, j]) # добавляем ошибку в список ошибок, если ребра между вершинами нет
#                 # raise RuntimeError("Clique contains unconnected vertices!") # если ребра нет — выкидываем ошибку
#     return errors # возвращаем ошибки


def transform_solution(solution) -> list:
    """
    Функция для преобразования ответа, чтобы номера шли не с 0, а с 1 и по порядку.\n
    Parameters:
        * solution: решение в формате (размер клики, [вершины в клике])\n
    Returns:
        * list: решение в формате (размер клики, [отсортированные инкрементированные вершины клики])
    """
    for i in range(solution[0]):
        solution[1][i] += 1
    return [solution[0], sorted(solution[1])]


def save_solution(solutions):
    """
    Функция для сохранения лучших ответов.\n
    Parameters:
        * solutions: словарь для тест-кейсов с решениями в формате (размер клики, [вершины в клике])\n
    Returns:
        * None: сохраняет файл
    """
    for dataset in solutions.keys(): # идём по тест-кейсам
        with open(f'solutions/{dataset}.csv', 'w', newline='') as file: # открываем файл для чистой записи
            writer = csv.writer(file) # создаём объект для записи
            writer.writerow([solutions[dataset]["clique_size"]]) # сохраняем размер клики
            writer.writerows([solutions[dataset]["clique"]]) # сохраняем вершины клики
    
    # сохранение таблицы в csv формате
    table = pd.DataFrame(data = [], columns=["Instance", "Time, sec", "Clique size", "Clique vertices"]) # создаём pandas таблицу
    for dataset in solutions.keys(): # идём по тест-кейсам
        testcase = pd.DataFrame(data = [[dataset, solutions[dataset]["time"], solutions[dataset]["clique_size"], solutions[dataset]["clique"]]], columns=["Instance", "Time, sec", "Clique size", "Clique vertices"]) # создаём "новую" таблицу для тест-кейса
        table = pd.concat([table, testcase], ignore_index=True) # объединяем таблицы
    table.to_csv("best_solutions.csv", index=False) # сохраняем в csv формате без индекса

## Считывание данных

In [4]:
# files = ["brock200_1", "brock200_2", "brock200_3", "brock200_4", "c-fat200-1", "c-fat200-2", "c-fat200-5", "c-fat500-1", "c-fat500-10", "c-fat500-2", "c-fat500-5", "C125.9", "gen200_p0.9_44", "gen200_p0.9_55",  "johnson8-2-4",  "johnson8-4-4", "johnson16-2-4", "hamming6-2", "hamming6-4", "hamming8-2", "hamming8-4", "keller4", "MANN_a9", "MANN_a27", "MANN_a45", "p_hat300-1", "p_hat300-2", "p_hat300-3", "san200_0.7_1", "san200_0.7_2", "san200_0.9_1", "san200_0.9_2", "san200_0.9_3", "sanr200_0.7"] # файлы, на которых должен быть протестирован код
# files = ["brock200_2", "brock200_3", "brock200_4", "johnson8-2-4", "johnson16-2-4", "MANN_a9", "hamming8-4", "keller4"] # файлы, на которых должен быть протестирован код
files = ["brock200_2", "brock200_3", "johnson8-2-4", "johnson16-2-4", "MANN_a9", "keller4"] # файлы, на которых должен быть протестирован код

In [5]:
data = {} 
# data - словарь вида 
# {"название датасета" : 
#     {"vertex_num": число вершин, 
#     "edge_num": число рёбер, 
#     "edges": 
#         {словарь вида вершина - set смежных ей вершин}
#     }
#  ...
# }

In [6]:
for file in files:
    data[file] = {"vertex_num": None, "edge_num": None, "edges": {}}
    with open("data/" + file + ".clq", "r") as f: # открываем файл для чтения
        for row in f: # проходим по строкам
            if row[0] == "c": # если строка начинается с буквы "c" - это комментарий, пропускае строку
                continue
            elif row[0] == "p": # если строка начинается с буквы "p" - это описание проблемы, берём из этой строки число вершин и рёбер (последние два числа)
                data[file]["vertex_num"], data[file]["edge_num"] = int(row.split()[-2]), int(row.split()[-1])
            elif row[0] == "e": # если строка начинается с буквы "p" - это вершины, между которыми есть ребро
                v1, v2 = int(row.split()[-2]) - 1, int(row.split()[-1]) - 1 # запоминаем вершины (-1, чтобы не было мороки с индексацией)

                # добавляем связь вершины v1 с v2
                if v1 not in data[file]["edges"].keys(): # если это первое упоминание вершины v1 - создадим для неё set с указанием v2
                    data[file]["edges"][v1] = {v2}
                elif v2 not in data[file]["edges"][v1]: # иначе - просто добавим v2 в set смежных вершин v1
                    data[file]["edges"][v1].add(v2)

                # аналогично, но относительно вершины v2
                if v2 not in data[file]["edges"].keys():
                    data[file]["edges"][v2] = {v1}
                elif v1 not in data[file]["edges"][v2]:
                    data[file]["edges"][v2].add(v1)
        data[file]["edges"] = dict(sorted(data[file]["edges"].items())) # отсортируем вершины в словаре (в set для ключа словаря вершины уже отсортированы)

In [7]:
data["johnson8-2-4"] # пример данных

{'vertex_num': 28,
 'edge_num': 210,
 'edges': {0: {5, 8, 9, 12, 13, 14, 17, 18, 19, 20, 23, 24, 25, 26, 27},
  1: {4, 7, 9, 11, 13, 14, 16, 18, 19, 20, 22, 24, 25, 26, 27},
  2: {3, 6, 9, 10, 13, 14, 15, 18, 19, 20, 21, 24, 25, 26, 27},
  3: {2, 7, 8, 11, 12, 14, 16, 17, 19, 20, 22, 23, 25, 26, 27},
  4: {1, 6, 8, 10, 12, 14, 15, 17, 19, 20, 21, 23, 25, 26, 27},
  5: {0, 6, 7, 10, 11, 14, 15, 16, 19, 20, 21, 22, 25, 26, 27},
  6: {2, 4, 5, 11, 12, 13, 16, 17, 18, 20, 22, 23, 24, 26, 27},
  7: {1, 3, 5, 10, 12, 13, 15, 17, 18, 20, 21, 23, 24, 26, 27},
  8: {0, 3, 4, 10, 11, 13, 15, 16, 18, 20, 21, 22, 24, 26, 27},
  9: {0, 1, 2, 10, 11, 12, 15, 16, 17, 20, 21, 22, 23, 26, 27},
  10: {2, 4, 5, 7, 8, 9, 16, 17, 18, 19, 22, 23, 24, 25, 27},
  11: {1, 3, 5, 6, 8, 9, 15, 17, 18, 19, 21, 23, 24, 25, 27},
  12: {0, 3, 4, 6, 7, 9, 15, 16, 18, 19, 21, 22, 24, 25, 27},
  13: {0, 1, 2, 6, 7, 8, 15, 16, 17, 19, 21, 22, 23, 25, 27},
  14: {0, 1, 2, 3, 4, 5, 15, 16, 17, 18, 21, 22, 23, 24, 27},
  15

# Реализация алгоритма

## Эвристики для поиска начального решения

In [8]:
def randomized_greedy_max_clique(edges:dict, iterations=10) -> tuple:
    """
    Функция для получения рандомизированного решения задачи о максимальной клике.\n
    Parameters:
        * edges: словарь смежных вершинам вершин
        * iterations: через сколько попыток без улучшения решения выходить из алгоритма\n
    Returns:
        * tuple: данные о найденной клике в формате (размер найденной клики, [вершина в клике 1, ..., вершина в клике k])
    """
    original_candidates = set(edges.keys()) # set вершин (изначально все являются кандидатами в клику)
    original_candidates_degrees = [len(edges[v]) for v in original_candidates] # создаём список степеней вершин (индекс - номер вершины, так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)

    attempts = 0 # текущее число попыток
    best_clique = [] # текущая лучшая клика

    while attempts < iterations: # запускаем алгоритм, пока число попыток без изменения результата не превысит счётчик iterations
        clique = [] # создаём "пустую" клику
        candidates = original_candidates.copy() # копируем всех кандидатов
        while len(candidates) != 0: # пока есть кандидаты — пытаемся добавить их в клику
            candidates_degrees = [original_candidates_degrees[i] for i in candidates] # пересчитываем степени кандидатов (оставляем степени только рассматриваемых вершин) для итерациии случайного выбора
            
            v = random.choices(population=list(candidates), weights=candidates_degrees, k=1)[0] # случайным образом выбираем вершину в клику в соответствии с её степенью (чем больше степень относительно других вершин — тем выше вероятность) (переводим candidates в список для случайного выбора)
            clique.append(v) # добавляем её в клику
            
            candidates = candidates.intersection(edges[v]) # среди кандидитов оставляем только тех, кто смежен со всеми вершинами в текущей клике (итеративно этот список постоянно уменьшается с добавлением новых вершин в клику)

        if len(clique) > len(best_clique): # если нашли новую лучшую клику, то запоминаем её
            best_clique = clique.copy()
            attempts = 0 # обнуляем число итераций без улучшения решения
        else:
            attempts += 1 # увеличиваем число итераций без улучшения решения

    return len(best_clique), best_clique # возвращаем размер лучшей клики и её саму

## Branch and Bound алгоритм

### 1) Branch and Bound для задачи о максимальной клике (без использования cplex)

In [9]:
class BranchAndBound_clq():
    def __init__(self, edges: dict, heuristic_for_init_sol: callable) -> None: # инициализация модели для решения задачи
        """
        Конструктор для модели.\n
        Parameters:
            * edges: словарь смежных вершинам вершин
            * heuristic_for_init_sol: функция, что будет использоваться для нахождения первичного решения\n
        Returns:
            * None: создаёт модель
        """
        # вершины в алгоритм приходят с нулевой (а не с 1)
        self.heuristic_for_init_sol = heuristic_for_init_sol # эвристическая функция для поиска начального решения
        self.edges = edges # данные о изначальном графе (словарь смежных вершинам вершин)
        self.num_vertices = len(edges) # число вершин в графе
        self.best_clique = [] # текущая лучшая клика
        self.cur_clique = [] # текущая рассматриваемая клика

    def calc_initial_solution(self, data) -> None:
        """
        Функция для поиска начального решения.\n
        Parameters:
            * data: дополнительные параметры, что будут отправлены в функцию поиска начального решения\n
        Returns:
            * None: обновляет best_clique
        """
        self.best_clique = self.heuristic_for_init_sol(self.edges, *data) # запоминаем содержимое найденной клики (размер будем брать через len, так как это всё равно за O(1))

    
    def initialize_bnb(self) -> None:
        """
        Функция инициализации Branch and Bound алгоритма для точного поиска максимальной клики.\n
        Returns:
            * None: обновляет best_clique
        """
        #===================================== случайным образом перемешиваем вершины-кандидатов ==========================================
        # candidates = list(range(self.num_vertices)) # изначально кандидаты в клику — все возможные вершины
        # random.shuffle(candidates) # случайно перемешиваем кандидатов
        #------------------------------------- сортируем smallest degree last with remove + reverse ---------------------------------------
        candidates = [-1] * self.num_vertices # массив-заготовка под кандидатов (массив пока заполнен значением -1) (почему-то при создании candidates как numpy.array работа замедляется раз в 5)
        vertices_degrees = [len(self.edges[v]) for v in range(self.num_vertices)] # создаём список степеней вершин (индекс - номер вершины (-1), так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)
        for i in range(self.num_vertices): # просто итерируемся столько раз, сколько у нас вершин
            vertex = vertices_degrees.index(min(vertices_degrees)) # берём вершину с самой малой степенью
            candidates[i] = vertex # записываем вершину на i-ую позицию с начала (не с конца, так как тут рассматривается reverse версия smallest degree last with remove)
            vertices_degrees[vertex] = np.inf # ставим такую степень найденной вершине, чтобы она больше ни разу не была выбрана
            for v in self.edges[vertex]: # идём по смежным вершинам рассматриваемой
                vertices_degrees[v] -= 1 # понижаем их степени
        #==================================================================================================================================

        self.recursive_bnb(candidates) # запускаем рекурсивный BnB алгоритм (DFS search)
    
    
    def recursive_bnb(self, candidates) -> None:
        """
        Функция для запуска точного алгоритма Branch and Bound.\n
        Parameters:
            * candidates: возможные кандидаты для добавления в клику\n
        Returns:
            * None: обновляет best_clique
        """

        if len(candidates) == 0: # если кандидатов не осталось — мы в листе BnB алгоритма
            if len(self.cur_clique) > len(self.best_clique): # если клика в листе получилась лучше — обновляем лучшую
                self.best_clique = self.cur_clique.copy() # запоминаем содержимое клики
            return # выходим из рекурсии, так как кандидатов не осталось
        
        if len(self.cur_clique) + len(candidates) <= len(self.best_clique): # проверка на UB (перестаём рассматривать ветку, если её ответ в любом случае не улучшит максимальную клику)
            return
        
        for i in range(len(candidates)): # идём по возможным кандидатам (ветвление)
            new_candidates = []
            for j in range(i+1, len(candidates)): # идём по оставшимся кандидатам
                # оставляем в новых кандидатах только те вершины, у которых есть ребро с добавляемой вершиной "i" (с вершинами в текущей клике у кандидатов уже проверено наличие ребра)
                if candidates[j] in self.edges[candidates[i]]: # проверяем, есть ли ребро между добавляемым кандидатом i и его "кандидатом по соседству" j
                    new_candidates.append(candidates[j]) # если ребро есть, значит эту вершину j мы можем добавить на следующей итерации алгоритма (j останется в кандидатах на следующем уровне DFS)
            self.cur_clique.append(candidates[i]) # добавляем вершину i в клику
            self.recursive_bnb(new_candidates) # рекурсивно вызываем BnB для оставшихся кандидатов (идём вглубь)
            self.cur_clique.remove(candidates[i]) # убираем добавленную вершину в клику для перехода к следующему (соседнему) элементу в DFS рекурсии
    

    def get_best_solution(self) -> tuple:
        """
        Функция, возвращающая лучшее найденное решение.\n
        Returns:
            * tuple: решение вида (размер клики, [вершины в клике])
        """
        return len(self.best_clique), self.best_clique

#### Тестирование

In [10]:
# solutions = {} 
# # словарь для ответов алгоритма
# # {"название датасета" : 
# #     {"clique_size": размер клики,
# #      "clique": [вершины, входящие в клику],
# #      "time": время на подсчёт
# #     }
# # }

In [11]:
# for dataset in tqdm(data.keys()): # идём по тест-кейсам
# # for dataset in ["johnson8-2-4", "brock200_2", "brock200_3"]:
#     print(dataset)
#     time_start = time.time() # замеряем время начала выполнения
#     for i in range(runs): # делаем runs запусков для усреднения времени
#         bnb_model = BranchAndBound_clq(edges=data[dataset]["edges"], heuristic_for_init_sol=randomized_greedy_max_clique) # создаём объект для модели
#         bnb_model.calc_initial_solution([iterations]) # запускаем поиск начального решения с передачей следующих параметров - (iterations - число запусков рандомизированного алгоритма поиска клики)
#         print(f"эвристическое решение {bnb_model.get_best_solution()}")
#         bnb_model.initialize_bnb() # запускаем Branch and Bound алгоритм
#         print(f"BnB решение {bnb_model.get_best_solution()}")
#     time_end = time.time() - time_start # считаем, сколько работал алгоритм
#     solution = bnb_model.get_best_solution() # берём полученное решение у модели
#     check_solution(edges=data[dataset]["edges"], solution=solution) # проверка решения
#     # # print("original solution", sol)
#     solution = transform_solution(solution) # сортирует вершины клики в порядке возрастания их номера и возвращает нумерацию с единицы
#     # print("transformed solution", sol)
#     solutions[dataset] = {"clique_size": solution[0], "clique": solution[1], "time": time_end/runs} # добавление

In [12]:
# solutions

### 2) Branch and Bound общего назначения (с использованием cplex)

In [32]:
class BranchAndBound():
    def __init__(self, edges: dict, heuristic_for_init_sol: callable, check_solution: callable, vars_lb: int=0, vars_ub: int=1) -> None: # инициализация модели для решения задачи
        """
        Конструктор для модели.\n
        Parameters:
            * edges: словарь смежных вершинам вершин
            * heuristic_for_init_sol: функция, что будет использоваться для нахождения первичного решения
            * check_solution: функция для проверки правильности решения (должна возвращать ошибки решения в виде массива; если решение правильно — этот список должен быть пуст)
            * vars_lb: минимальное значение, что могут принимать все переменные
            * vars_ub: максимальное значение, что могут принимать все переменные\n
        Returns:
            * None: создаёт модель
        """
        self.model = Model(name="Linear Program") # cplex модель для задачи
        self.heuristic_for_init_sol = heuristic_for_init_sol # эвристическая функция для поиска начального решения
        self.check_solution = check_solution # функция для проверки правильности решения
        self.edges = edges # данные о изначальном графе (словарь смежных вершинам вершин)
        self.num_vertices = len(edges) # число вершин в графе

        self.best_solution = [0, []] # текущее лучшее решение в формате [значение целевой функции, [вершина в клике 1, ..., вершина в клике k]]

        self.vars = [0]*self.num_vertices # список с переменными задачи
        for var_id in range(self.num_vertices): # добавляем в модель непрерывные переменные
            self.vars[var_id] = self.model.continuous_var(name=f"x{var_id}", lb=vars_lb, ub=vars_ub) # добавляем непрерывные переменные-вершины x{var_id}, var_id от 0 до число_вершин_в_графе-1 (-1 из-за нумерации с нуля), что могут принимать значение в диапазоне [vars_lb, vars_ub]

        self.model.set_objective("max", sum(self.vars)) # добавляем целевую функцию в cplex модель - максимизация суммы переменных


    def calc_initial_solution(self, data) -> None:
        """
        Функция для поиска начального решения.\n
        Parameters:
            * data: дополнительные параметры, что будут отправлены в функцию поиска начального решения\n
        Returns:
            * None: обновляет "лучшее" решение
        """
        self.best_solution = self.heuristic_for_init_sol(self.edges, *data) # находим начальное решение эвристикой (запоминаем размер найденной клики и её вершины)
    

    def recursive_bnb(self, prev_solution=None, eps=0.000001) -> None:
        """
        Функция для запуска точного алгоритма Branch and Bound.\n
        Parameters:
            * prev_solution: решение с предыдущего шага
            * eps: с какой погрешностью считать, что переменная целая\n
        Returns:
            * None: обновляет best_clique
        """
        solution = self.model.solve() # получаем решение задачи от cplex solver-а
        solution = [solution.objective_value, solution.get_all_values()] # преобразовываем solution в формат [значение целевой функции, [вершины в клике]]

        if solution[0] <= self.best_solution[0]: # проверка на UB (перестаём рассматривать ветку, если она в любом случае не улучшит целевую функцию)
            return

        if prev_solution == solution: # если ответ с предыдущего шага равен текущему (значение целевой функции и всех переменных) — выходим из рекурсии, так как такое возможно лишь при неправильном ответе (не прошёл check_solution), но который измениться не может в связи с наложенными ограничениями
            print("trapped") # DEBUG
            return

        # vars_fraction = [] # список для дробных переменных
        # for var_id, var_value in enumerate(solution[1]): # идём по переменным
        #     if not ((1.0 - eps <= var_value <= 1.0 + eps) or (-eps <= var_value <= eps)): # проверяем, дробная ли вершина
        #         vars_fraction.append(var_id) # если дробная — добавляем номер переменной в список дробных

        # ищем дробную вершину с наибольшим значением, по которой будем ветвиться (ближайшая к наличию в клике)
        var_fraction = [-1, -1] # дробная переменная в формате [номер переменной, её значение], по которой будем ветвиться (если таковая будет в решении)
        for var_id, var_value in enumerate(solution[1]): # идём по переменным
            if not ((1.0 - eps <= var_value <= 1.0 + eps) or (-eps <= var_value <= eps)): # проверяем, дробная ли переменная
                if var_value > var_fraction[1]: # если переменная — дробная и её значение самое наибольшее, то будем ветвиться по ней
                    var_fraction = [var_id, var_value] # запоминаем выбранную переменную

        if var_fraction[0] > -1: # если была найдена дробная переменная
            # print(f"fraction: {var_fraction}") # DEBUG
            self.model.add_constraint(self.vars[var_fraction[0]] >= 1, ctname=f"x{var_fraction[0]}>=1") # добавляем ограничение для переменной >=1 (в клике)
            self.recursive_bnb(prev_solution=solution) # вызываем BnB на ветку, где дробная переменная >= 1 (с запоминанием текущего ответа)
            self.model.remove_constraint(f"x{var_fraction[0]}>=1") # убираем добавленное ограничение после того, как DFS пройдёт по его ветке

            self.model.add_constraint(self.vars[var_fraction[0]] <= 0, ctname=f"x{var_fraction[0]}<=0") # добавляем ограничение для переменной <=0 (не в клике)
            self.recursive_bnb(prev_solution=solution) # вызываем BnB на ветку, где дробная переменная <= 0 (с запоминанием текущего ответа)
            self.model.remove_constraint(f"x{var_fraction[0]}<=0") # убираем добавленное ограничение после того, как DFS пройдёт по его ветке
        else: # если дробных переменных нет — проверяем решение на корректность
            # print(f"no fraction") # DEBUG
            # solution[1] = [var_id for var_id, var_value in enumerate(solution[1]) if (1.0 - eps <= var_value <= 1.0 + eps)] # в solution заменяем переменные на вершины, что "находятся в клике"
            solution_ = [solution[0], [var_id for var_id, var_value in enumerate(solution[1]) if (1.0 - eps <= var_value <= 1.0 + eps)]] # в solution_ заменяем переменные на вершины, что "находятся в клике"
            # в [var_id for var_id, var_value in enumerate(solution[1]) if (1.0 - eps <= var_value <= 1.0 + eps)] пронумеровываем все вершины в решении и берём номера только тех вершин, у которых значение = 1 (состоят в клике)

            errors = self.check_solution(self.edges, solution_) # получаем список ошибок решения (если решение без ошибок, то check_solution должен вернуть пустой список)
            if len(errors) > 0: # если список с ошибками не пуст — добавляем их в ограничения и перезапускаем BnB
                # print("with errors") # DEBUG
                # print(errors) # DEBUG
                for error in errors: # идём по парам ошибочно связанных вершин (error имеет формат [переменная 1, переменная 2])
                    self.model.add_constraint(self.vars[error[0]] + self.vars[error[1]] <= 1, ctname=f"x{error[0]}+x{error[1]}<=1") # добавляем ограничение
                self.recursive_bnb(prev_solution=solution) # вызываем BnB для пересчёта текущего решения
            elif solution_[0] > self.best_solution[0]: # если значение целевой функции улучшилось (все переменные не дробные и решение корректное)
                self.best_solution = solution_.copy() # сохраняем полученное лучшее решение
            # else: # иначе — в решении нет ошибок # DEBUG
            #     # print("no errors") # DEBUG
            #     if solution_[0] > self.best_solution[0]: # если значение целевой функции улучшилось (все переменные не дробные и решение корректное) # DEBUG
            #         print("updating best +") # DEBUG
            #         self.best_solution = solution_.copy() # сохраняем полученное лучшее решение # DEBUG
            #     else: # DEBUG
            #         print("updating best -") # DEBUG
        return # выходим из рекурсии (возвращаемся на уровень выше в DFS)
        

    def get_best_solution(self) -> tuple:
        """
        Функция, возвращающая лучшее найденное решение.\n
        Returns:
            * tuple: решение вида (значение целевой функции, [вершина в клике 1, ..., вершина в клике k])
        """
        return self.best_solution

In [33]:
bnb_model = BranchAndBound(edges=data["keller4"]["edges"], heuristic_for_init_sol=randomized_greedy_max_clique, check_solution=check_solution)
bnb_model.calc_initial_solution([iterations]) # запускаем поиск начального решения с передачей следующих параметров - (iterations - число запусков рандомизированного алгоритма поиска клики)
bnb_model.recursive_bnb()
best_solution = bnb_model.get_best_solution()
best_solution = transform_solution(best_solution)
best_solution

KeyboardInterrupt: 

In [178]:
bnb_model = BranchAndBound(edges=data[dataset]["edges"], heuristic_for_init_sol=randomized_greedy_max_clique, check_solution=check_solution)
bnb_model.model.solve()
# bnb_model.model.print_solution()
# bnb_model.get_best_solution() # вывод лучшего решения

# bnb_model.model.solve().get_value_dict({"x0":0.0, "x1":0.0}, keep_zeros=False) # не работает

# bnb_model.cur_solution # текущее решение

docplex.mp.solution.SolveSolution(obj=171,values={x0:1,x1:1,x2:1,x3:1,x4..

In [180]:
bnb_model.model.add_constraint(bnb_model.cur_solution[1][0] + bnb_model.cur_solution[1][1] + bnb_model.cur_solution[1][2] <= 1, ctname="x0+x1+x2<=1")

docplex.mp.LinearConstraint[x0+x1+x2<=1](x0+x1+x2,LE,1)

In [181]:
bnb_model.model.add_constraint(bnb_model.cur_solution[1][0] + bnb_model.cur_solution[1][1] + bnb_model.cur_solution[1][2] >= 0, ctname="x0+x1+x2>=0") # меняем ограничение

docplex.mp.LinearConstraint[x0+x1+x2>=0](x0+x1+x2,GE,0)

In [183]:
# поиск ограничений
# bnb_model.model.get_constraint_by_name("x0+x1+x2")
# bnb_model.model.get_constraint_by_index(0)
# bnb_model.model.get_constraint_by_index(1)
bnb_model.model.remove_constraint("x0+x1+x2<=1")

In [187]:
bnb_model.model.get_constraint_by_index(0)

docplex.mp.LinearConstraint[x0+x1+x2>=0](x0+x1+x2,GE,0)

In [184]:
bnb_model.model.solve()
bnb_model.model.solution.objective_value

171.0

In [165]:
bnb_model.model.get_var_by_index(0).solution_value

0

In [118]:
# sol.get_all_values() # возвращает все переменные в виде списка
[i for i,v in enumerate(sol.get_all_values()) if v == 1] # элементы со значением 1

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170]

In [114]:
for v in bnb_model.model.iter_variables():
    print(v," = ",v.solution_value)

x0  =  1.0
x1  =  1.0
x2  =  1.0
x3  =  1.0
x4  =  1.0
x5  =  1.0
x6  =  1.0
x7  =  1.0
x8  =  1.0
x9  =  1.0
x10  =  1.0
x11  =  1.0
x12  =  1.0
x13  =  1.0
x14  =  1.0
x15  =  1.0
x16  =  1.0
x17  =  1.0
x18  =  1.0
x19  =  1.0
x20  =  1.0
x21  =  1.0
x22  =  1.0
x23  =  1.0
x24  =  1.0
x25  =  1.0
x26  =  1.0
x27  =  1.0
x28  =  1.0
x29  =  1.0
x30  =  1.0
x31  =  1.0
x32  =  1.0
x33  =  1.0
x34  =  1.0
x35  =  1.0
x36  =  1.0
x37  =  1.0
x38  =  1.0
x39  =  1.0
x40  =  1.0
x41  =  1.0
x42  =  1.0
x43  =  1.0
x44  =  1.0
x45  =  1.0
x46  =  1.0
x47  =  1.0
x48  =  1.0
x49  =  1.0
x50  =  1.0
x51  =  1.0
x52  =  1.0
x53  =  1.0
x54  =  1.0
x55  =  1.0
x56  =  1.0
x57  =  1.0
x58  =  1.0
x59  =  1.0
x60  =  1.0
x61  =  1.0
x62  =  1.0
x63  =  1.0
x64  =  1.0
x65  =  1.0
x66  =  1.0
x67  =  1.0
x68  =  1.0
x69  =  1.0
x70  =  1.0
x71  =  1.0
x72  =  1.0
x73  =  1.0
x74  =  1.0
x75  =  1.0
x76  =  1.0
x77  =  1.0
x78  =  1.0
x79  =  1.0
x80  =  1.0
x81  =  1.0
x82  =  1.0
x83  =  1.0
x8

# Testing

In [50]:
def smallest_degree_last_with_remove_v3(edges: dict): # на вход - словарь смежных вершине вершин
    # сортируем smallest degree last with remove
    edges_sorted = {}

    vertices = edges.keys() # список вершин
    vertices_sorted = np.full(shape=(len(vertices)), fill_value=-1, dtype=np.int16) # заготовка под номера вершин (массив пока заполнен значением -1)

    vertices_degrees = [len(edges[v]) for v in vertices] # создаём список степеней вершин (индекс - номер вершины (-1), так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)
    
    for i in range(len(vertices)-1, -1, -1): # просто итерируемся столько раз, сколько у нас вершин (но с конца: len(vertices)-1 - максимальный индекс, с которого будем идти (-1 чтобы не выйти за рамки массива); -1 по какой индекс идём не включительно (то есть до нуля); -1 шаг)
        vertex = vertices_degrees.index(min(vertices_degrees)) # берём вершину с самой малой степенью
        vertices_sorted[i] = vertex # записываем вершину на i-ую позицию с конца
        vertices_degrees[vertex] = np.inf # ставим такую степень найденной вершине, чтобы она больше ни разу не была выбрана
        for v in edges[vertex]: # идём по смежным вершинам
            vertices_degrees[v] -= 1 # понижаем их степени (v-1 так как  нумерация в vertices_degrees идёт с нуля, а не с единицы)

    for vertex in vertices_sorted: # идём по отсортированному smallest degree last with remove массиву вершин
        edges_sorted[vertex] = edges[vertex] # собираем обратно edges

#    print("edges_sorted", edges_sorted)
    return edges_sorted

In [62]:
edges = {0:{1,2,3,5}, 1:{0,2,3}, 2:{0,1,3}, 3:{0,1,2,4}, 4:{3,6,7,8,9,10}, 5:{0,11,12,13,14,15}, 6:{4}, 7:{4}, 8:{4}, 9:{4}, 10:{4}, 11:{5}, 12:{5}, 13:{5}, 14:{5}, 15:{5}}
smallest_degree_last_with_remove_v3(edges)

{3: {0, 1, 2, 4},
 2: {0, 1, 3},
 1: {0, 2, 3},
 0: {1, 2, 3, 5},
 5: {0, 11, 12, 13, 14, 15},
 15: {5},
 14: {5},
 13: {5},
 12: {5},
 11: {5},
 4: {3, 6, 7, 8, 9, 10},
 10: {4},
 9: {4},
 8: {4},
 7: {4},
 6: {4}}